In [ ]:
from time import sleep

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession. \
    builder. \
    appName("Data Sources"). \
    master("local"). \
    config("spark.jars", "data/jars/postgresql-42.2.19.jar"). \
    config("spark.sql.legacy.timeParserPolicy", "LEGACY"). \
    getOrCreate()


# config("spark.sql.warehouse.dir", "data/warehouse"). \
    

In [ ]:
cars_df = spark.read.json("data/sources/cars").cache()
cars_df.show()

assert(cars_df.count() != 0)

american_cars_df = cars_df.\
    filter(col("Origin") == "Japan").\
    select(col("Name"))

american_cars_df.show()


In [ ]:
# store as a Spark table dont write data to storage EXTERNAL TABLE
#  DataFrame => SQL metastore
print("DataFrame => SQL metastore = EXTERNAL TABLE")
cars_df.createOrReplaceTempView("cars")

In [ ]:
# run SQL queries on top of DFs known to Spark under a certain name
american_cars_df_v2 = spark.sql("SELECT Name FROM cars WHERE Origin = 'Japan'")
american_cars_df_v2.show()

In [ ]:
# DROPPING EXTERNAL TABLE DOES NOT DELETE DATA, ONLY IN METASTORE
print("DROPPING EXTERNAL TABLE")
spark.sql("DROP TABLE cars")
spark.sql("SELECT Name FROM cars WHERE Origin = 'Japan'")
american_cars_df.show()
cars_df_again = spark.read.json("data/cars")
cars_df_again.show()

In [ ]:
# store DFs as Spark tables (files known to Spark) to Spark storage HDFS
#  DataFrame => SQL metastore + Spark storage
print("SAVING TO MANAGED TABLE")
cars_df.\
    write.\
    mode("overwrite").\
    saveAsTable("cars_managed_table")
# parquet("data/parquet"). \
#     save()

NameError: name 'cars_df' is not defined

In [ ]:
print("READ FROM MANAGED STORAGE 1")
assert(cars_df.count() != 0)
american_cars_df_v2 = spark.sql("SELECT * FROM cars_managed_table")
assert(american_cars_df_v2.count() != 0)
american_cars_df_v2.show()


In [ ]:
print("READ FROM MANAGED STORAGE 2")
cars_managed_df = spark.table("cars_managed_table")
# cars_managed_df = spark.read.table("cars_managed_table")
assert (cars_managed_df.count() != 0)
cars_managed_df.show()


In [ ]:
print("DROPPING MANGED TABLE")
spark.sql("DROP TABLE cars_managed_table")

In [ ]:
print("DDL DML INSIDE TABLE")
spark.sql("CREATE SCHEMA test")

spark.sql("CREATE TABLE test.students (name VARCHAR(64), address VARCHAR(64)) USING PARQUET PARTITIONED BY (student_id INT)")

spark.sql("INSERT INTO test.students VALUES('Bob Brown', '456 Taylor St, Cupertino', 222222),('Cathy Johnson', '789 Race Ave, Palo Alto', 333333)")

ddl_demo_df = spark.sql("SELECT * FROM test.students")
print("DDL_DEMO_DF TABLE")
assert (ddl_demo_df.count() != 0)
ddl_demo_df.show()


In [ ]:
print("DROPPING MANGED TABLE")
spark.sql("DROP TABLE test.students")

 # https://spark.apache.org/docs/latest/sql-ref-syntax-dml-insert-into.html

In [ ]:
driver = "org.postgresql.Driver"
url = "jdbc:postgresql://localhost:5432/spark"
user = "docker"
password = "docker"


def read_table(table_name):
    return spark.read. \
        format("jdbc"). \
        option("driver", driver). \
        option("url", url). \
        option("user", user). \
        option("password", password). \
        option("dbtable", "public." + table_name). \
        load()


exersices
1. show all employees and their max salary over time
2. show all employees who were never managers
3. * for every employee, find the difference between their salary (current/latest) and the max salary of their department (departments table)


In [ ]:
employees_df = read_table("employees")
salaries_df = read_table("salaries")
dept_managers_df = read_table("dept_manager")
dept_emp_df = read_table("dept_emp")
departments_df = read_table("departments")

# save table names
employees_df.createOrReplaceTempView("employees")
salaries_df.createOrReplaceTempView("salaries")
dept_managers_df.createOrReplaceTempView("dept_manager")
dept_emp_df.createOrReplaceTempView("dept_emp")
departments_df.createOrReplaceTempView("departments")
